In [6]:
!pip install pychorus

Defaulting to user installation because normal site-packages is not writeable


In [7]:
# import necessary libraries 
import os 
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')

In [8]:
# read new data table with downloaded full songs path
pwd = os.getcwd()
data_path = os.path.join(pwd,'extracted_chorus.csv')
data = pd.read_csv(data_path)
df = data.copy()

In [9]:
df[df['song_path'].isna()] #check for null paths

,Artist,Title,Label,song_path,extracted_chorus_path


In [10]:
full_songs_path = list(df['song_path']) # list of full songs downloaded path 
chorus_extract_path = 'chorus_extract' # path to save extracted chorus
extracted_songs = [basename.replace('.wav','') for basename in os.listdir(chorus_extract_path)] # basename/ song name of already extracted song to avoid repitition in chorus extraction 
# extracted_songs

In [11]:
from pychorus import find_and_output_chorus # import chorus ectractor lib
not_extrated = [] # list to track not extracted choruses 

In [12]:
# helper function to extract 15s choruses from songs as required in paper 
def extract_chorus(path_list, output_path):
    for path in path_list:
        basename = os.path.basename(str(path)).replace('.mp3', '')
        save_path = f'{output_path}/{basename}.wav' # output path as .wav extension 
        if basename in extracted_songs:
            print('song already exists')
        else:
            chorus = find_and_output_chorus(str(path), str(save_path), 15)
            if chorus == None:
                not_extrated.append(basename)        

In [13]:
extract_chorus(full_songs_path,chorus_extract_path) #extract all choruses 

song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists
song already exists


NoBackendError: 

In [14]:
len(not_extrated) # check total number of not extracted songs 

0

In [15]:
extracted_song_basename = [song.replace('.wav','') for song in os.listdir('chorus_extract')] # get extracteds songs basename
extracted_song_list = [] # list for songs from table present in successful extraction list 
for song in df['song_path']:
    basename = os.path.basename(song).replace('.mp3','')
    if  basename in extracted_song_basename:
        extracted_song_list.append(f'chorus_extract/{basename}.wav') # append successfuly extracted songs to list
    else:
        extracted_song_list.append('n/a') # append not available if extracted song does not exist 

In [16]:
len(extracted_song_list)

792

In [18]:
df = df.assign(extracted_chorus_path=extracted_song_list)# add new exteacted chorus paths to data table 
df

,Artist,Title,Label,song_path,extracted_chorus_path
0,The Weeknd,Blinding Lights,1,popular/Blinding Lights.mp3,chorus_extract/Blinding Lights.wav
1,Olivia Rodrigo,Good 4 U,1,popular/Good 4 U.mp3,chorus_extract/Good 4 U.wav
2,Olivia Rodrigo,Drivers License,1,popular/Drivers License.mp3,chorus_extract/Drivers License.wav
3,Lil Nas X,Montero (Call Me By Your Name),1,popular/Montero (Call Me By Your Name).mp3,chorus_extract/Montero (Call Me By Your Name).wav
4,BTS,Butter,1,popular/Butter.mp3,chorus_extract/Butter.wav
...,...,...,...,...,...
787,Gwen Stefani,Crash,0,unpopular/Crash.mp3,chorus_extract/Crash.wav
788,Gwen Stefani,Luxurious,0,unpopular/Luxurious.mp3,chorus_extract/Luxurious.wav
789,Rihanna,If It's Lovin' That You Want,0,unpopular/If It's Lovin' That You Want.mp3,n/a
790,Alicia Keys,Unbreakable,0,unpopular/Unbreakable.mp3,chorus_extract/Unbreakable.wav


In [19]:
df[df['extracted_chorus_path']=='n/a'] # check for non available extracts path from list of chorus path

,Artist,Title,Label,song_path,extracted_chorus_path
19,Drake,What's Next,1,popular/What's Next.mp3,n/a
35,Morgan Wallen,Chasin' You,1,popular/Chasin' You.mp3,n/a
47,Luke Combs,Even Though I'm Leaving,1,popular/Even Though I'm Leaving.mp3,n/a
50,Luke Combs,Lovin' On You,1,popular/Lovin' On You.mp3,n/a
60,Shawn Mendes,If I Can't Have You,1,popular/If I Can't Have You.mp3,n/a
...,...,...,...,...,...
756,Maroon 5,Won't Go Home Without You,0,unpopular/Won't Go Home Without You.mp3,n/a
770,Avril Lavigne,When You're Gone,0,unpopular/When You're Gone.mp3,n/a
773,Kanye West,Can't Tell Me Nothing,0,unpopular/Can't Tell Me Nothing.mp3,n/a
786,Carrie Underwood,Don't Forget To Remember Me,0,unpopular/Don't Forget To Remember Me.mp3,n/a


In [20]:
#save table to csv file 
df.to_csv('extracted_chorus_incomplete.csv', index=False) # dataset with incomplete extracted choruses 

In [21]:
data1 = pd.read_csv('extracted_chorus_incomplete.csv')
data1.isna().sum() # get total number of non available extracted paths 

Artist                    0
Title                     0
Label                     0
song_path                 0
extracted_chorus_path    83
dtype: int64

In [22]:
data1 = data1.dropna() # drop rows with non available extract path 
data1.isna().sum()

Artist                   0
Title                    0
Label                    0
song_path                0
extracted_chorus_path    0
dtype: int64

In [ ]:
#save table to csv file 
data1.to_csv('extracted_chorus.csv', index=False) 